In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

In [4]:
cd /home/jupyter/

/home/jupyter


In [5]:
df_train = pd.read_parquet('intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip')
df_test = pd.read_parquet('intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip')

Build a XGBoost model to classify event segments

In [11]:
X_train = df_train.loc[:,~df_train.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_train = df_train['veranst_segment']

X_test = df_test.loc[:,~df_test.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_test = df_test['veranst_segment']

In [12]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [13]:
xgboost_classifier = xgboost.XGBClassifier()
xgboost_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [18]:
# save the model to disk
xbg_filename = 'intellizenz-model-training/models/2022-09-29/xgb_classifier_model_with_inkasso_time_series.sav'
pickle.dump(xgboost_classifier, open(xbg_filename, 'wb'))

In [19]:
y_pred = xgboost_classifier.predict(X_test)

Evaluation

In [20]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

xgboost_accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(xgboost_accuracy)

[[438354  14101 119135]
 [107190  32176 108766]
 [135289  14506 424275]]
              precision    recall  f1-score   support

           0       0.64      0.77      0.70    571590
           1       0.53      0.13      0.21    248132
           2       0.65      0.74      0.69    574070

    accuracy                           0.64   1393792
   macro avg       0.61      0.55      0.53   1393792
weighted avg       0.63      0.64      0.61   1393792

894805
